# Extracción 

In [23]:
import pandas as pd
import requests

def cargar_datos_desde_github(url):
    # Obtener los datos del archivo Excel desde GitHub
    respuesta = requests.get(url)
    # Asegurarse de que la petición es exitosa
    respuesta.raise_for_status()
    
    # Cargar los datos en un DataFrame de pandas
    from io import BytesIO
    datos = pd.read_excel(BytesIO(respuesta.content))
    
    return datos

# URL del archivo Excel en formato raw en GitHub
url = 'https://github.com/Shamirchurio/Clasificaciontexto/raw/main/Data%20clasificada.xlsx'

# Cargar datos
try:
    datos = cargar_datos_desde_github(url)
    print("Datos cargados correctamente:")
    print(datos.head())  # Muestra las primeras filas del DataFrame
except Exception as e:
    print("Hubo un problema al cargar los datos:", e)

# Suponiendo que quieras trabajar con la columna 'Calificacion_analisis'
# Aquí puedes continuar con tu análisis o manipulación de esos datos


Datos cargados correctamente:
        Nombre proceso_x                                        Indicador_x  \
0  Servicio Farmacéutico           Porcentaje de medicamentos devueltos ***   
1  Servicio Farmacéutico          Porcentaje de medicamentos devueltos ****   
2  Servicio Farmacéutico  Porcentaje de cobertura en validación farmacéu...   
3  Servicio Farmacéutico           Porcentaje de medicamentos devueltos ***   
4      Gestión Ambiental  Porcentaje de residuos peligrosos administrati...   

  Código Indicador_x Unidad de medida_x Periodicidad_x Finalidad_x  \
0       FAR0-04-03-1         Porcentaje        Mensual   Minimizar   
1       FAR0-04-03-3         Porcentaje        Mensual   Minimizar   
2       FAR1-03-01-1         Porcentaje        Mensual   Maximizar   
3       FAR0-04-03-1         Porcentaje        Mensual   Minimizar   
4       GAM0-04-06-3         Porcentaje        Mensual   Minimizar   

                                           Fórmula_x  \
0  (( Número de me

In [25]:
datos.columns

Index(['Nombre proceso_x', 'Indicador_x', 'Código Indicador_x',
       'Unidad de medida_x', 'Periodicidad_x', 'Finalidad_x', 'Fórmula_x',
       'Variables_x', 'Análisis_x', 'Semaforo', 'Calificación',
       'Periodo medido', 'Calificacion_analisis'],
      dtype='object')

# Transformación

In [30]:
mapeo = {
    2: 'malo',
    1: 'bueno',
    0: 'medio'
}

# Reemplazar los valores numéricos por las etiquetas usando el diccionario
datos['Calificacion_analisis'] = datos['Calificacion_analisis'].replace(mapeo)

In [31]:
datos.Calificacion_analisis

0        malo
1        malo
2        malo
3        malo
4        malo
        ...  
4728    bueno
4729    bueno
4730    medio
4731    bueno
4732    bueno
Name: Calificacion_analisis, Length: 4733, dtype: object

# Conexión API GPT

In [18]:
!pip install openai==1.2.0

  Attempting uninstall: openai
    Found existing installation: openai 1.1.1
    Uninstalling openai-1.1.1:
      Successfully uninstalled openai-1.1.1


In [3]:
from openai import OpenAI
client = OpenAI(api_key="api")

response = client.chat.completions.create(
  model="gpt-3.5-turbo",
  messages=[
    {
      "role": "user",
      "content": ""
    }
  ],
  temperature=1,
  max_tokens=256,
  top_p=1,
  frequency_penalty=0,
  presence_penalty=0
)

In [32]:
def clasificar_calidad_analisis(analisis):
    # Ejemplos de clasificación para guiar al modelo
    prompt = f"""
    Clasifica los siguientes análisis de indicadores:
    
    
    Análisis:"{datos.Análisis_x}" Clasificación:{datos.Calificacion_analisis} 
    Análisis: "{analisis}" Clasificación:
    """
    
    respuesta = client.chat.completions.create(
        
        model="gpt-3.5-turbo",  # Cambia según tu acceso y necesidades
        messages=[
            {
              "role": "user",
              "content": prompt
            }
          ],
        max_tokens=60
        )
    
    # Extraer y devolver la clasificación de la respuesta
    return respuesta.choices[0].message.content

# Ejemplo de uso
analisis = "La variación del indicador este mes no muestra una mejora significativa, aunque se mantiene estable."
clasificacion = clasificar_calidad_analisis(analisis)
print("Clasificación:", clasificacion)

Clasificación: bueno


# Text

In [37]:
analisis = "baja operación"
clasificacion = clasificar_calidad_analisis(analisis)
print("Clasificación:", clasificacion)

Clasificación: Clasificación: malo
